## Задача:
    На основе данных notMNIST_small построить модель, определяющую букву на картинке 
    
### Цель:
    - Построить модель, определяющую букву на изображении 28х28.
    - Начать изучать keras, tensorflow
    
### Данные:
    - notMNIST_small - 10 директорий с изображениями букв.
    
### План выполнения:
     1. Подготовка данных
     2. Построение и обучение модели нейронной сети
     3. Вывод

In [1]:
import numpy as np
import os
import sys
import pickle
from collections import defaultdict
import matplotlib.pyplot as plt
from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, SeparableConv2D, MaxPool2D, Dropout, Dense, Flatten, Activation
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from keras.utils import np_utils
from keras.optimizers import SGD, Adadelta, Adam, rmsprop

from hyperas import optim
from hyperas.distributions import choice
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope 

from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline

Using TensorFlow backend.


In [2]:
# константы
DIR_ROOT = './notMNIST_small'
DIRS = os.listdir(DIR_ROOT)
print(DIRS)
# функция для конвертирования float to int из интервала hp.uniform hyperopt
@scope.define
def to_int(x):
    return int(x)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']


#### 1. Подготовка данных

In [3]:
# считаем данные картинок и составим массивы:
#    - признаков 28х28 (размер изображения)
#    - таргет переменная
X = np.empty((0,28,28), float)
y = np.empty((0,1), int)

err = {}

dict_target = {i:j for i,j in enumerate(DIRS)}

for idl, dr in enumerate(DIRS):
    
    files = os.listdir(os.path.join(DIR_ROOT, dr))
    for i, file in enumerate(files):
        try:
            im = plt.imread(os.path.join(DIR_ROOT, dr, file))
            X = np.concatenate( ( X, [im]), axis=0)
            y = np.concatenate( ( y, [[idl]]), axis=0)
        except Exception as ex:
            print( dr, file, ex)
            err[file] = dr



A RGVtb2NyYXRpY2FCb2xkT2xkc3R5bGUgQm9sZC50dGY=.png read past end of file
F Q3Jvc3NvdmVyIEJvbGRPYmxpcXVlLnR0Zg==.png read past end of file


In [4]:
# разделим на трейн и тест
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,test_size=0.2,random_state=41)

In [5]:
# сохраним, чтобы потом не повторять все вышесделанные операции
with open('data_small.pickle', 'wb') as f:
    pickle.dump([dict_target, X_train, X_test, y_train, y_test], f)


In [6]:
with open('data_small.pickle', 'rb') as f:
    [dict_target, X_train, X_test, y_train, y_test] = pickle.load(f)

In [7]:
# преобразуем двумерный массив признаков в одномерный
X_train = X_train.reshape(X_train.shape[0], 784)
X_test = X_test.reshape(X_test.shape[0], 784)
# приведем все значения признаков к интервалу [0,1], 
# т.к. tensorflow принимает принимает значения только в этом интервале
X_train /= 255
X_test /= 255
# разделим на трейн и валидационные выборки, для обучения
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,stratify=y_train,test_size=0.25,random_state=41)
# преобразуем векторы классов в бинарные матрицы классов
Y_train = np_utils.to_categorical(y_train, len(dict_target))
Y_test = np_utils.to_categorical(y_test, len(dict_target))
Y_val = np_utils.to_categorical(y_val, len(dict_target))

#### 2.  Построение и обучение модели нейронной сети

In [8]:
# создадим переменную с пераметрами для hyperopt
params = {  'choice': hp.choice('num_layers',
                    [ {'layers':'two', },
                    {'layers':'three',
                    'units3':  scope.to_int(hp.uniform('units3', 64,1024)), 
                    'dropout3': hp.uniform('dropout3', .25,.75)}
                    ]),

            'units1':  scope.to_int(hp.uniform('units1', 64,1024)),
            'units2':  scope.to_int(hp.uniform('units2', 64,1024)),

            'dropout1': hp.uniform('dropout1', .25,.75),
            'dropout2': hp.uniform('dropout2',  .25,.75),

            'batch_size' :  scope.to_int(hp.uniform('batch_size', 32,256)),

            'nb_epochs' :  100,
            'optimizer': hp.choice('optimizer',['adadelta','adam','rmsprop']),
            'activation': 'relu'
        }



In [9]:
# функция нейронной сети для hyperopt
def func_fit(params):  
    print ('Params: ', params)
    model = Sequential()
    model.add(Dense(output_dim=params['units1'], input_dim = 784)) 
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout1']))

    model.add(Dense(output_dim=params['units2'], init = "glorot_uniform")) 
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout2']))

    if params['choice']['layers']== 'three':
        model.add(Dense(output_dim=params['choice']['units3'], init = "glorot_uniform")) 
        model.add(Activation(params['activation']))
        model.add(Dropout(params['choice']['dropout3']))    

    model.add(Dense(10))
    model.add(Activation('softmax'))
    model.compile(loss='binary_crossentropy', optimizer=params['optimizer'],  metrics=['acc'])

    model.fit(X_train, Y_train, nb_epoch=100, batch_size=params['batch_size'], verbose = 0)

    pred_auc =model.predict_proba(X_val, batch_size =params['batch_size'], verbose = 0)
    acc = roc_auc_score(Y_val, pred_auc)
    score = model.evaluate(X_val, Y_val, verbose=0)
    print('AUC:', acc)
    print('Score: ', score)
    return {'loss': -acc, 'status': STATUS_OK}



In [10]:
# запустим fmin(hyperopt) для определения наилучщих параметров
trials=Trials()
best = fmin(func_fit, space=params, algo=tpe.suggest, max_evals=100, trials=trials)
print('best: ')
print(best)

Params:                                                                                                                
{'activation': 'relu', 'batch_size': 170, 'choice': {'dropout3': 0.48003107432909753, 'layers': 'three', 'units3': 359}, 'dropout1': 0.2627876084802413, 'dropout2': 0.6626937610515793, 'nb_epochs': 100, 'optimizer': 'rmsprop', 'units1': 448, 'units2': 961}
  0%|                                                                            | 0/100 [00:00<?, ?it/s, best loss: ?]

C:\Users\And\.conda\envs\proba\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=784, units=448)`
  """



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


C:\Users\And\.conda\envs\proba\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=961, kernel_initializer="glorot_uniform")`
  if __name__ == '__main__':

C:\Users\And\.conda\envs\proba\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=359, kernel_initializer="glorot_uniform")`
  

C:\Users\And\.conda\envs\proba\lib\site-packages\ipykernel_launcher.py:22: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.



Instructions for updating:
Use tf.cast instead.
AUC:                                                                                                                   
0.9939043788591384                                                                                                     
Score:                                                                                                                 
[0.08454585944207368, 0.985260352528143]                                                                               
Params:                                                                                                                
{'activation': 'relu', 'batch_size': 227, 'choice': {'layers': 'two'}, 'dropout1': 0.49828797974883604, 'dropout2': 0.6085910312511453, 'nb_epochs': 100, 'optimizer': 'adam', 'units1': 931, 'units2': 111}
  1%|▍                                               | 1/100 [01:22<2:16:35, 82.78s/it, best loss: -0.9939043788591384]

...

AUC:                                                                                                                   
0.9843562691944567                                                                                                     
Score:                                                                                                                 
[0.07636530026917822, 0.9760747713463329]                                                                              
Params:                                                                                                                
{'activation': 'relu', 'batch_size': 146, 'choice': {'dropout3': 0.5067951381990292, 'layers': 'three', 'units3': 196}, 'dropout1': 0.5645600504598012, 'dropout2': 0.3564609063705403, 'nb_epochs': 100, 'optimizer': 'adam', 'units1': 870, 'units2': 532}
 98%|██████████████████████████████████████████████ | 98/100 [1:35:22<02:16, 68.29s/it, best loss: -0.9960467140488285]

C:\Users\And\.conda\envs\proba\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=784, units=870)`
  """

C:\Users\And\.conda\envs\proba\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=532, kernel_initializer="glorot_uniform")`
  if __name__ == '__main__':

C:\Users\And\.conda\envs\proba\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=196, kernel_initializer="glorot_uniform")`
  

C:\Users\And\.conda\envs\proba\lib\site-packages\ipykernel_launcher.py:22: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.



AUC:                                                                                                                   
0.9956127225372381                                                                                                     
Score:                                                                                                                 
[0.05550326484408142, 0.9866755695623135]                                                                              
Params:                                                                                                                
{'activation': 'relu', 'batch_size': 185, 'choice': {'layers': 'two'}, 'dropout1': 0.6387262338332245, 'dropout2': 0.5681208407489512, 'nb_epochs': 100, 'optimizer': 'adam', 'units1': 626, 'units2': 646}
 99%|██████████████████████████████████████████████▌| 99/100 [1:36:39<01:11, 71.07s/it, best loss: -0.9960467140488285]

C:\Users\And\.conda\envs\proba\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=784, units=626)`
  """

C:\Users\And\.conda\envs\proba\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=646, kernel_initializer="glorot_uniform")`
  if __name__ == '__main__':

C:\Users\And\.conda\envs\proba\lib\site-packages\ipykernel_launcher.py:22: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.



AUC:                                                                                                                   
0.995284401662223                                                                                                      
Score:                                                                                                                 
[0.04844721587982839, 0.9863818480589679]                                                                              
100%|██████████████████████████████████████████████| 100/100 [1:37:41<00:00, 68.11s/it, best loss: -0.9960467140488285]
best: 
{'batch_size': 164.8875630005353, 'dropout1': 0.7341277311739184, 'dropout2': 0.40382383673725447, 'dropout3': 0.3463663478464305, 'num_layers': 1, 'optimizer': 1, 'units1': 888.5874958243104, 'units2': 978.3996468326723, 'units3': 630.2093083651171}


In [11]:
# обучим модель с подобранными параметрами и приверим на тестовых данных
model = Sequential()
model.add(Dense(output_dim=707, input_dim = int(28*28))) 
model.add(Activation('relu'))
model.add(Dropout(.59))

model.add(Dense(output_dim=594, init = "glorot_uniform")) 
model.add(Activation('relu'))
model.add(Dropout(.49))


model.add(Dense(output_dim=971, init = "glorot_uniform")) 
model.add(Activation('relu'))
model.add(Dropout(.6))    

model.add(Dense(10))
model.add(Activation('softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])

history = model.fit(X_train, Y_train, nb_epoch=50, batch_size=255, verbose = 1)


pred_auc =model.predict_proba(X_test, batch_size =255, verbose = 0)
acc = roc_auc_score(Y_test, pred_auc)
score = model.evaluate(X_test, Y_test, verbose=0)
print('AUC:', acc)
print('Score: ', score)

C:\Users\And\.conda\envs\proba\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=784, units=707)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\And\.conda\envs\proba\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=594, kernel_initializer="glorot_uniform")`
  import sys
C:\Users\And\.conda\envs\proba\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=971, kernel_initializer="glorot_uniform")`
  if sys.path[0] == '':
C:\Users\And\.conda\envs\proba\lib\site-packages\ipykernel_launcher.py:20: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/50
11234/11234 [==============================] - ETA: 4:23 - loss: 0.3251 - acc: 0.900 - ETA: 39s - loss: 0.3250 - acc: 0.900 - ETA: 16s - loss: 0.3247 - acc: 0.90 - ETA: 9s - loss: 0.3243 - acc: 0.9000 - ETA: 5s - loss: 0.3233 - acc: 0.900 - ETA: 3s - loss: 0.3209 - acc: 0.900 - ETA: 1s - loss: 0.3166 - acc: 0.900 - ETA: 0s - loss: 0.3087 - acc: 0.900 - 7s 588us/step - loss: 0.3039 - acc: 0.9007
Epoch 2/50
11234/11234 [==============================] - ETA: 0s - loss: 0.2325 - acc: 0.908 - ETA: 0s - loss: 0.2147 - acc: 0.911 - ETA: 0s - loss: 0.2002 - acc: 0.917 - ETA: 0s - loss: 0.1914 - acc: 0.922 - ETA: 0s - loss: 0.1835 - acc: 0.925 - ETA: 0s - loss: 0.1763 - acc: 0.929 - ETA: 0s - loss: 0.1695 - acc: 0.933 - ETA: 0s - loss: 0.1640 - acc: 0.936 - 0s 42us/step - loss: 0.1633 - acc: 0.9366
Epoch 3/50
11234/11234 [==============================] - ETA: 0s - loss: 0.1364 - acc: 0.954 - ETA: 0s - loss: 0.1260 - acc: 0.953 - ETA: 0s - loss: 0.1255 - acc: 0.955 - ETA: 0s - loss:

11234/11234 [==============================] - ETA: 0s - loss: 0.0360 - acc: 0.988 - ETA: 0s - loss: 0.0350 - acc: 0.987 - ETA: 0s - loss: 0.0330 - acc: 0.988 - ETA: 0s - loss: 0.0332 - acc: 0.989 - ETA: 0s - loss: 0.0346 - acc: 0.988 - ETA: 0s - loss: 0.0344 - acc: 0.988 - ETA: 0s - loss: 0.0351 - acc: 0.988 - ETA: 0s - loss: 0.0349 - acc: 0.988 - 0s 40us/step - loss: 0.0348 - acc: 0.9885
Epoch 42/50
11234/11234 [==============================] - ETA: 0s - loss: 0.0324 - acc: 0.989 - ETA: 0s - loss: 0.0372 - acc: 0.987 - ETA: 0s - loss: 0.0377 - acc: 0.987 - ETA: 0s - loss: 0.0377 - acc: 0.987 - ETA: 0s - loss: 0.0365 - acc: 0.987 - ETA: 0s - loss: 0.0349 - acc: 0.988 - ETA: 0s - loss: 0.0355 - acc: 0.988 - ETA: 0s - loss: 0.0353 - acc: 0.988 - 0s 42us/step - loss: 0.0353 - acc: 0.9883
Epoch 43/50
11234/11234 [==============================] - ETA: 0s - loss: 0.0314 - acc: 0.989 - ETA: 0s - loss: 0.0326 - acc: 0.988 - ETA: 0s - loss: 0.0340 - acc: 0.987 - ETA: 0s - loss: 0.0342 - acc:

#### 3. Вывод:
        Построил и обучил модель нейронной сети. Использовал hyperopt для подбора параметров. В итоге построил нейронную сеть с двумя скрытыми слоями Dense. Получилась неплохая модель со score: 0.9856, roc_auc: 0.9955.